# Interacting with PyCodeGPT

In [2]:
import json
prompts = []
anss = []
# domain = 'TextEditing'
# domain = 'ASTMatcher'
origin_domain = 'Flight'
path = f'/home/scruple/NLP/datasets/CERT/{origin_domain}/data/valid/output.jsonl'
with open(path, 'r') as valid:
    for val in valid.readlines():
        res = json.loads(val)
        prompt, ans = res['content'].split('\nANSWER:\n')
        prompt += '\nANSWER:\n'
        print(prompt+ans)
        prompts.append(prompt)
        anss.append(ans)
        # print(ans)



LIBRARY:
Flight

QUESTION:
Information on flights from boston to washington


ANSWER:
AtomicRowPredSet(AtomicRowPred(EQ_DEPARTS(CITY(boston),ANY(),ANY(),ANY(),ANY()),EQ_ARRIVES(CITY(washington),ANY(),ANY(),ANY(),ANY())))


LIBRARY:
Flight

QUESTION:
Kindly give me the type of aircraft used to fly from atlanta to denver


ANSWER:
PROJECT(AtomicColSet(COL_AIRCRAFT_TYPE()),AtomicRowPredSet(AtomicRowPred(EQ_DEPARTS(CITY(atlanta),ANY(),ANY(),ANY(),ANY()),EQ_ARRIVES(CITY(denver),ANY(),ANY(),ANY(),ANY()))))


LIBRARY:
Flight

QUESTION:
What type of aircraft is used flying from atlanta to denver before 12 noon


ANSWER:
PROJECT(AtomicColSet(COL_AIRCRAFT_TYPE()),AtomicRowPredSet(AtomicRowPred(EQ_DEPARTS(CITY(atlanta),Unit_Time_Set(MATCH_TIME_LT(TIME(12))),ANY(),ANY(),ANY()),EQ_ARRIVES(CITY(denver),ANY(),ANY(),ANY(),ANY()))))


LIBRARY:
Flight

QUESTION:
On eastern flight 825 from atlanta to denver can you tell me what type of aircraft used before 12 noon


ANSWER:
PROJECT(AtomicColSet(COL_AIRC

In [3]:
# %%writefile /content/PyCodeGPT/eval_human_eval.py
import os
import re
import argparse
import time
from tqdm import tqdm

from transformers import pipeline, set_seed
from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers.pipelines.base import Pipeline


# from human_eval.data import write_jsonl, read_problems

def load_generation_pipe(model_name_or_path: str, gpu_device: int=0):
    model = AutoModelForCausalLM.from_pretrained(model_name_or_path)
    tokenizer = AutoTokenizer.from_pretrained(model_name_or_path)

    pipe = pipeline(
        'text-generation',
        model=model,
        tokenizer=tokenizer,
        device=gpu_device
    )

    print("load generation pipeline from {} over, vocab size = {}, eos id = {}, gpu device = {}.".format(
        model_name_or_path, len(tokenizer), tokenizer.eos_token_id, gpu_device)
    )

    return pipe

def extract_function_block(string):
    return re.split("\nclass|\ndef|\n#|\n@|\nprint|\nif", string)[0].rstrip()

def run_code_generation(pipe, prompt, num_completions=1, **gen_kwargs):
    set_seed(123)

    code_gens = pipe(prompt,
        num_return_sequences=num_completions,
        **gen_kwargs
    )

    return [extract_function_block(code_gen["generated_text"][len(prompt):]) for code_gen in code_gens]

class print_time:
    def __init__(self, desc):
        self.desc = desc

    def __enter__(self):
        print(self.desc)
        self.t = time.time()

    def __exit__(self, type, value, traceback):
        print(f'{self.desc} took {time.time()-self.t:.02f}s')

def evaluate_on_human_eval(
    model_name_or_path: str,
    temperature: float,
    top_p: float,
    num_samples_per_task: int,
    max_new_tokens: int,
    gpu_device: int,
    output_dir: str,
    ) -> str:

    pipe: Pipeline = load_generation_pipe(model_name_or_path, gpu_device=gpu_device)
    eval_name = f"human_eval.t{temperature}.p{top_p}.l{max_new_tokens}.n{num_samples_per_task}"

    if output_dir is None:
        if os.path.exists(model_name_or_path):
            output_dir = model_name_or_path
        else:
            raise ValueError("Output dir can't be null if you are not evaluation a local model.")

    os.makedirs(output_dir, exist_ok=True)
    saved_path = os.path.join(output_dir, f"{eval_name}.samples.jsonl")

    gen_kwargs = {
        "do_sample": True,
        "temperature": temperature,
        "max_new_tokens": max_new_tokens,
        "top_p": top_p,
        "top_k": 0,
        "pad_token_id": pipe.tokenizer.pad_token_id if pipe.tokenizer.pad_token_id else pipe.tokenizer.eos_token_id,
        "eos_token_id": pipe.tokenizer.eos_token_id
    }

    # problems = read_problems()
    samples = []
    generate_batch_size = min(50, num_samples_per_task)

    bos_token = pipe.tokenizer.bos_token if pipe.tokenizer.bos_token else pipe.tokenizer.eos_token

    # for task_id in tqdm(problems):
    #     # Strip operation is important as new tokenizer will not treat '\n' as a independent token
    #     prompt = problems[task_id]["prompt"].strip()

    #     for _ in range(num_samples_per_task // generate_batch_size):
    cnt = 0
    # prompts = ['I would like to findthe cheapest flightfrom Baltimore to Atlanta']
    # anss = ['22']
    for prompt, ans in zip(prompts, anss):
        with print_time('sampling'):
            print('=' * 100)
            input_prompt = prompt
            print("input_prompt: \n" + input_prompt)
            gen_results = run_code_generation(pipe, input_prompt, num_completions=generate_batch_size, **gen_kwargs)
            print('=' * 100)
            print("generated code: \n")
            flag = False
            
            # ans = ans.strip()
            # for gen_result in gen_results:
            #     print('-' * 100)
            #     gen_result = gen_result.strip()
            #     if gen_result == "":
            #         continue
            #     # if check_result(gen_result, ans, domain):
            #     if gen_result.lower() == ans.lower():
            #         flag = True
            #     print("generated:", gen_result)
            #     print("answer   :", ans)
            cnt+=check_result(origin_domain, gen_results, ans)[0]
                
            # if flag:
            #     cnt+=1
        # break
    print(f'{domain}: ')
    print(cnt, len(anss))
    print(cnt/len(anss))
          # samples.append(dict(task_id=task_id, completion=gen_result))
    
    # write_jsonl(saved_path, samples)
    # print("Run generation over, save {} samples to {}.".format(len(samples), saved_path))
# domain = 'TextEditing_hisyn'
# domain = 'ASTMatcher'
# domain = 'ASTMatcher_hisyn'
# domain = 'Flight'
domain = 'Flight_hisyn'
def check_result(domain, expr_list, code):
    if domain == 'TextEditing' or domain == 'Flight':
        for i in range(len(expr_list)):
            expr_list[i] = expr_list[i].replace('1st', '1').replace('2nd', '2').replace('3rd', '3').replace('4th', '4').replace(
                '5th', '5').replace('6th', '6').replace('7th', '7').replace('8th', '8').replace('10th', '10').replace('1()', '1').replace('80th', '80')


        expr_lower_list = [n.lower().replace(' ', '') for n in expr_list]
        code_compare = code.lower().replace(' ', '').replace('\n', '')

        print('expr lower list:', expr_lower_list)
        print('correct code:   ', code_compare)

        result = code_compare in expr_lower_list
        print(result)

        if not result:
            if 'start()' in code_compare:
                if (code_compare.replace('start()', 'position(before(linetoken()),all())')) in expr_lower_list:
                    result = True
                elif (code_compare.replace('start()', 'position(before(string()),all())')) in expr_lower_list:
                    result = True
        res = 0
        if result:
            res = 1

        return [res, expr_lower_list, code_compare]

    elif domain == 'ASTMatcher':

        expr_lower_list = [n.lower().replace(' ', '').replace('_c', '') for n in expr_list]
        code_compare = code.lower().replace(' ', '').replace('\n', '')

        print('expr lower list:', expr_lower_list)
        print('correct code:   ', code_compare)


        result = code_compare in expr_lower_list
        res = 0
        if result:
            res = 1

        if res:
            print(True)
        else:
            print(False)

        return [res, expr_lower_list, code_compare]
evaluate_on_human_eval(
    model_name_or_path=f'/home/scruple/NLP/datasets/CERT/{domain}/experiments/normal_models/1x2x1x512-5e-4-0.1-1K_100_00K_5K-RS_1.0_0.5_1.0',
    temperature=0.8,
    top_p=0.95,
    num_samples_per_task=1,
    max_new_tokens=100,
    gpu_device=0,
    output_dir='results/',
)


load generation pipeline from /home/scruple/NLP/datasets/CERT/Flight_hisyn/experiments/normal_models/1x2x1x512-5e-4-0.1-1K_100_00K_5K-RS_1.0_0.5_1.0 over, vocab size = 32000, eos id = 0, gpu device = 0.
sampling
input_prompt: 

LIBRARY:
Flight

QUESTION:
Information on flights from boston to washington


ANSWER:

generated code: 

expr lower list: ['atomicrowpredset(atomicrowpred(eq_departs(city(boston),any(),any(),any(),any()),eq_arrives(city(washington),any(),any(),any(),any())))']
correct code:    atomicrowpredset(atomicrowpred(eq_departs(city(boston),any(),any(),any(),any()),eq_arrives(city(washington),any(),any(),any(),any())))
True
sampling took 1.00s
sampling
input_prompt: 

LIBRARY:
Flight

QUESTION:
Kindly give me the type of aircraft used to fly from atlanta to denver


ANSWER:

generated code: 

expr lower list: ['project(atomiccolset(col_aircraft_type()),extract_row_min_f(col_fare(),atomicrowpredset(atomicrowpred(eq_departs(city(ck_city),any(),any(),any(),any()),eq_arrives(

/home/scruple/miniconda3/envs/cert/lib/python3.7/site-packages/transformers/pipelines/base.py:910: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  UserWarning,


generated code: 

expr lower list: ['atomicrowpredset(atomicrowpred(eq_departs(city(pittsburgh),any(),any(),any(),any()),eq_arrives(city(atlanta),any(),any(),daynum(fifteenth),month(september))))']
correct code:    atomicrowpredset(atomicrowpred(eq_departs(city(pittsburgh),any(),any(),daynum(fourth),month(september)),eq_arrives(city(atlanta),any(),any(),any(),any())))
False
sampling took 0.29s
sampling
input_prompt: 

LIBRARY:
Flight

QUESTION:
I would like to plan a flight on american airlines from boston to denver


ANSWER:

generated code: 

expr lower list: ['atomicrowpredset(atomicrowpred(eq_departs(city(boston),any(),any(),any(),any()),eq_arrives(city(denver),any(),any(),any(),any())),eq_airlines(airlines(american)))']
correct code:    atomicrowpredset(atomicrowpred(eq_departs(city(boston),any(),any(),any(),any()),eq_arrives(city(denver),any(),any(),any(),any())),eq_airlines(airlines(american)))
True
sampling took 0.30s
sampling
input_prompt: 

LIBRARY:
Flight

QUESTION:
Hello i 

In [9]:
print('TextEdting:', 13/93)
print('ASTMatcher:', 3/20)
print('Flight:', 33/107)

TextEdting: 0.13978494623655913
ASTMatcher: 0.15
Flight: 0.308411214953271


: 

TextEditing: 
23 71
0.323943661971831

TextEditing_hisyn: 
24 71
0.3380281690140845

Flight: 
19 66
0.2878787878787879

Flight_hisyn: 
18 66
0.2727272727272727

Flight_hisyn: (paraphrase)
22 66
0.3333333333333333

Flight_hisyn: (expr_list)
24 66
0.36363636363636365
<!-- Flight_hisyn: 
17 66
0.25757575757575757 -->

In [ ]:
valid_size: 71
TextEditing: 0.323
TextEditing_hisyn: 0.338

valid_size: 66
Flight: 0.287
Flight_hisyn: 0.272